In [2]:
# Importing necessary libraries and modules
import os
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
from utils import create_conditions
import matplotlib.pyplot as plt

In [3]:
# Load data
cases = pd.read_csv("./data/covid/africa_daily_time_series_cases.csv")
deaths = pd.read_csv("./data/covid/africa_daily_time_series_deaths.csv")
recoveries = pd.read_csv("./data/covid/africa_daily_time_series_recovered.csv")

In [28]:
# cases[(cases.iloc[:, 3:13].columns > 0).any(axis=1)]
first_month_cases = cases.loc[:, cases.iloc[:,3:13].columns.values]
first_month_cases[(first_month_cases > 0).all(1)]



1/22/20    0
1/23/20    0
1/24/20    0
1/25/20    0
1/26/20    0
1/27/20    0
1/28/20    0
1/29/20    0
1/30/20    0
1/31/20    0
dtype: int64

In [83]:
dates = pd.to_datetime(cases.iloc[:, 3:,].columns.values).to_series()
month_end_dates = dates.groupby(dates.index.month).apply(lambda x: x.index.max())
month_end_dates.apply(lambda x: print(x.strftime('%m/%d/%y')))
# pd.to_datetime(month_end_dates, format='%d/%m/%Y')

31/01/20
29/02/20
31/03/20
27/04/20


1    None
2    None
3    None
4    None
dtype: object